In [1]:
import io
from tqdm import tqdm
from itertools import islice
import numpy as np

def load_vectors(fname, limit):
  fin = io.open(fname, 'r', encoding = 'utf-8', newline = '\n', errors = 'ignore')
  n, d = map(int, fin.readline().split())
  data = {}
  for line in tqdm(islice(fin, limit), total = limit):
    tokens = line.rstrip().split(' ')
    data[tokens[0]] = np.array(list(map(float, tokens[1:])))
  return data

vecs = load_vectors('crawl-300d-2M.vec', 100000)   

FileNotFoundError: [Errno 2] No such file or directory: 'crawl-300d-2M.vec'

In [ ]:
def get_k_nearest_neighbors(vec, k):
  return list(zip(*sorted(list(map(lambda key: (np.linalg.norm(vec - vecs[key]), key), vecs.keys())))))[1][:k]

print(get_k_nearest_neighbors(vecs['Paris'], 20))
print(get_k_nearest_neighbors(vecs['brother'], 20))

In [ ]:
get_k_nearest_neighbors(vecs['Paris'] - vecs['France'] + vecs['Germany'], 1)


In [ ]:
get_k_nearest_neighbors(vecs['brother'] - vecs['man'] + vecs['woman'], 1)

In [ ]:
get_k_nearest_neighbors(vecs['king'] - vecs['man'] + vecs['woman'], 5)


In [ ]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset = 'train', categories = categories, shuffle = True, random_state = 42)
twenty_test = fetch_20newsgroups(subset='test', categories = categories,shuffle = True, random_state = 42)
X_train = twenty_train.data
y_train = twenty_train.target
X_test = twenty_test.data
y_test = twenty_test.target


In [ ]:
vecs = load_vectors('crawl-300d-2M.vec', 2000000)   


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)
logreg = LogisticRegression(solver = 'liblinear', multi_class = 'ovr', random_state = 1)

logreg.fit(X_train_tfidf, y_train)
y_pred = logreg.predict(X_test_tfidf)
print(accuracy_score(y_test, y_pred))


In [ ]:
print(X_train_tfidf.shape)


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 300, random_state = 1)
X_train_tfidf_pca = pca.fit_transform(X_train_tfidf.todense())
X_test_tfidf_pca = pca.transform(X_test_tfidf.todense())

logreg.fit(X_train_tfidf_pca, y_train)
y_pred_pca = logreg.predict(X_test_tfidf_pca)
print(accuracy_score(y_test, y_pred_pca))


In [ ]:
import numpy as np

zero = sum(vecs.values()) / len(vecs)
def text2vec(text):
  words = text.split()
  return sum(list(map(lambda w: np.array(list(vecs.get(w, zero))), words))) / len(words)


In [ ]:
X_train_vec = list(map(lambda text: text2vec(text), X_train))
X_test_vec = list(map(lambda text: text2vec(text), X_test))


In [ ]:
logreg.fit(X_train_vec, y_train)
y_pred_vec = logreg.predict(X_test_vec)
print(accuracy_score(y_test, y_pred_vec))



In [ ]:
dim = 300
vocab = np.zeros((len(tfidf.vocabulary_.keys()), dim))
for key in tqdm(tfidf.vocabulary_.keys()):
  vocab[tfidf.vocabulary_[key]] = vecs.get(key, zero)

In [ ]:
X_train_weighted = X_train_tfidf.dot(vocab)
X_test_weighted = X_test_tfidf.dot(vocab)


In [ ]:
logreg.fit(X_train_weighted, y_train)
y_pred_weighted = logreg.predict(X_test_weighted)
print(accuracy_score(y_test, y_pred_weighted))


In [ ]:
import pandas as pd

reviews = pd.read_csv('imdb_master.csv', encoding = 'latin-1')['review']


In [ ]:
from gensim.models import Word2Vec

sentences = list(map(lambda text: text.split(), reviews))
model = Word2Vec(sentences, min_count = 1, seed = 1, workers = 1)


In [ ]:
wv = model.wv


In [ ]:

print(wv.similar_by_word('Paris'))
print()
print(wv.similar_by_word('brother'))
print()
print(wv.similar_by_vector(wv['king'] - wv['man'] + wv['woman'], 1))